# Feature Extraction

In most time-series machine learning tasks, raw signal data is not used directly to train models. Instead, we perform **feature extraction** to transform segments, or "windows", of the time-series into a set of informative features that better represent the underlying patterns. This process converts a sequence of data points into a single row of features that a model can learn from.

**IMPORTANT:** All feature extractor classes in this toolkit are designed to operate on **pre-windowed data**. This means you **must** first use the `Windowing` class to prepare your time-series. The workflow is always:

1.  **Windowing Step:** Convert the raw time-series `DataFrame` into a new `DataFrame` where each row represents a single window.
2.  **Feature Extraction Step:** Pass the windowed `DataFrame` to one of the feature extractor classes.

In this notebook, we will demonstrate the three primary feature extraction methods available in the toolkit, following this two-step process.

In [ ]:
import pandas as pd
from pathlib import Path

from ThreeWToolkit.feature_extraction import (
    extract_wavelet_features,
    extract_statistical_features,
    extract_exponential_statistics_features,
)

from ThreeWToolkit.core.base_preprocessing import WindowingConfig
from ThreeWToolkit.core.base_feature_extractor import (
    WaveletConfig,
    StatisticalConfig,
    EWStatisticalConfig,
)
from ThreeWToolkit.preprocessing._data_processing import Windowing

from ThreeWToolkit.dataset import ParquetDataset
from ThreeWToolkit.core.base_dataset import EventPrefixEnum, ParquetDatasetConfig

## Loading 3W Dataset

In [2]:
dataset_path = Path("../../dataset")
event_types = [EventPrefixEnum.REAL]
ds_config = ParquetDatasetConfig(
    path=dataset_path, clean_data=True, event_type=event_types
)  # load all files, target defaults to 'class'
ds = ParquetDataset(ds_config)
len(ds)

[ParquetDataset] Dataset found at ../../dataset
[ParquetDataset] Validating dataset integrity...
[ParquetDataset] Dataset integrity check passed!


1119

In [3]:
X_raw = ds[0]["signal"]
y_raw = ds[0]["label"]

X_raw

,ABER-CKGL,ABER-CKP,ESTADO-DHSV,ESTADO-M1,ESTADO-M2,ESTADO-PXO,ESTADO-SDV-GL,ESTADO-SDV-P,ESTADO-W1,ESTADO-W2,...,P-JUS-CKGL,P-JUS-CKP,P-MON-CKP,P-PDG,P-TPT,QGL,T-JUS-CKP,T-MON-CKP,T-PDG,T-TPT
timestamp,,,,,,,,,,,,,,,,,,,,,
2018-09-12 02:58:00,-0.635461,-0.068467,0.867921,0.414652,-0.681653,-0.094347,0.0,0.312558,-1.537219,-0.563169,...,0.230992,0.0,0.051273,0.0,-0.887248,0.0,-0.713450,0.0,0.0,0.294243
2018-09-12 02:58:01,-0.635461,-0.068467,0.867921,0.414652,-0.681653,-0.094347,0.0,0.312558,-1.537219,-0.563169,...,0.230992,0.0,0.044067,0.0,-0.886955,0.0,-0.713422,0.0,0.0,0.294318
2018-09-12 02:58:02,-0.635461,-0.068467,0.867921,0.414652,-0.681653,-0.094347,0.0,0.312558,-1.537219,-0.563169,...,0.230992,0.0,0.036862,0.0,-0.886663,0.0,-0.713395,0.0,0.0,0.294390
2018-09-12 02:58:03,-0.635461,-0.068467,0.867921,0.414652,-0.681653,-0.094347,0.0,0.312558,-1.537219,-0.563169,...,0.230992,0.0,0.029657,0.0,-0.887053,0.0,-0.713367,0.0,0.0,0.294465
2018-09-12 02:58:04,-0.635461,-0.068467,0.867921,0.414652,-0.681653,-0.094347,0.0,0.312558,-1.537219,-0.563169,...,0.230992,0.0,0.022452,0.0,-0.887443,0.0,-0.713340,0.0,0.0,0.294537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-09-18 14:59:56,-0.635461,-0.068699,0.867921,0.414652,-0.681653,-0.094347,0.0,0.312558,-1.537219,-0.563169,...,0.154657,0.0,0.134754,0.0,-0.834359,0.0,-0.841294,0.0,0.0,0.289674
2018-09-18 14:59:57,-0.635461,-0.068698,0.867921,0.414652,-0.681653,-0.094347,0.0,0.312558,-1.537219,-0.563169,...,0.154657,0.0,0.137187,0.0,-0.834749,0.0,-0.841484,0.0,0.0,0.289601
2018-09-18 14:59:58,-0.635461,-0.068698,0.867921,0.414652,-0.681653,-0.094347,0.0,0.312558,-1.537219,-0.563169,...,0.154657,0.0,0.139621,0.0,-0.835139,0.0,-0.841674,0.0,0.0,0.289526


## Wavelet Feature Extraction

The `ExtractWaveletFeatures` class uses a signal processing technique called the Stationary Wavelet Transform (SWT). This method decomposes the signal within each window into different frequency components, which can often capture patterns that are invisible to standard statistical measures.

For each level of decomposition, two sets of coefficients are generated:
* **Approximation Coefficients (A):** These capture the low-frequency, underlying trend of the signal. Think of it as a "smoothed" version of the signal within the window.
* **Detail Coefficients (D):** These capture the high-frequency components, representing noise, spikes, and other abrupt changes. 
These features allow a model to differentiate between the general shape of a signal and its more noisy, high-frequency texture.

In [4]:
# Define parameters that will be shared between windowing and feature extraction
# The window_size for the wavelet transform is determined by its level
LEVEL = 3
WINDOW_SIZE = 2**LEVEL
OVERLAP = 0.875  # This is equivalent to a stride of 1 for a window of 8

# Configure and instantiate the Windowing step
# We use a 'boxcar' window because we don't want to change the signal
# before applying the wavelet transform.
windowing_config = WindowingConfig(
    window_size=WINDOW_SIZE, overlap=OVERLAP, window="boxcar"
)
windowing_step = Windowing(windowing_config)

# Run the windowing step on the raw data
windowed_x = windowing_step(X_raw)

# Align the labels (y)
# The Windowing class doesn't handle labels, so we must align them manually.
# The label for each window corresponds to the label at the end of that window in the original series.
step = int(WINDOW_SIZE * (1 - OVERLAP))
if step == 0:
    step = 1
end_of_window_indices = [
    i + WINDOW_SIZE - 1 for i in range(0, len(X_raw) - WINDOW_SIZE + 1, step)
]
aligned_y = y_raw.iloc[end_of_window_indices]

# Combine the windowed features and aligned labels into a single DataFrame
#  We reset the index to ensure they concatenate correctly.
windowed_data = pd.concat(
    [windowed_x.reset_index(drop=True), aligned_y.reset_index(drop=True)], axis=1
)

print("Shape of the pre-windowed data:", windowed_data.shape)
print("The data is now ready for the feature extraction step.")
windowed_data.head()

Shape of the pre-windowed data: (561714, 178)
The data is now ready for the feature extraction step.


,var1_t0,var1_t1,var1_t2,var1_t3,var1_t4,var1_t5,var1_t6,var1_t7,var2_t0,var2_t1,...,var22_t0,var22_t1,var22_t2,var22_t3,var22_t4,var22_t5,var22_t6,var22_t7,win,class
0,-0.635461,-0.635461,-0.635461,-0.635461,-0.635461,-0.635461,-0.635461,-0.635461,-0.068467,-0.068467,...,0.294243,0.294318,0.294390,0.294465,0.294537,0.294318,0.294095,0.294023,1,7
1,-0.635461,-0.635461,-0.635461,-0.635461,-0.635461,-0.635461,-0.635461,-0.635461,-0.068467,-0.068467,...,0.294318,0.294390,0.294465,0.294537,0.294318,0.294095,0.294023,0.293948,2,7
2,-0.635461,-0.635461,-0.635461,-0.635461,-0.635461,-0.635461,-0.635461,-0.635461,-0.068467,-0.068467,...,0.294390,0.294465,0.294537,0.294318,0.294095,0.294023,0.293948,0.294171,3,7
3,-0.635461,-0.635461,-0.635461,-0.635461,-0.635461,-0.635461,-0.635461,-0.635461,-0.068467,-0.068467,...,0.294465,0.294537,0.294318,0.294095,0.294023,0.293948,0.294171,0.294390,4,7
4,-0.635461,-0.635461,-0.635461,-0.635461,-0.635461,-0.635461,-0.635461,-0.635461,-0.068467,-0.068467,...,0.294537,0.294318,0.294095,0.294023,0.293948,0.294171,0.294390,0.294243,5,7


In [5]:
# Configure the feature extractor
wavelet_config = WaveletConfig(level=LEVEL, wavelet="haar")

# Instantiate the extractor
feature_extractor = extract_wavelet_features.ExtractWaveletFeatures(wavelet_config)
feature_extractor.is_windowed = True

# Run the feature extraction step by calling the instance
wavelet_features = feature_extractor(windowed_data)

# Display the results
print("Shape of the final extracted features:", wavelet_features.shape)
print(
    "\nColumns are named 'var<index>_<feature>', and the 'label' column is preserved."
)
wavelet_features.head()

Shape of the final extracted features: (561714, 154)

Columns are named 'var<index>_<feature>', and the 'label' column is preserved.


,var1_A3,var1_D3,var1_A2,var1_D2,var1_A1,var1_D1,var1_A0,var2_A3,var2_D3,var2_A2,...,var21_A1,var21_D1,var21_A0,var22_A3,var22_D3,var22_A2,var22_D2,var22_A1,var22_D1,var22_A0
0,-1.797355,-6.714585e-17,-1.270922,-1.503918e-17,-0.898677,-8.230210e-18,-0.635461,-0.193654,1.216413e-18,-0.136934,...,0.0,0.0,0.0,0.832402,-0.000156,0.588487,-3.684697e-04,0.415862,-0.000051,0.294023
1,-1.797355,-6.714585e-17,-1.270922,-1.503918e-17,-0.898677,-8.230210e-18,-0.635461,-0.193655,-1.241565e-07,-0.136935,...,0.0,0.0,0.0,0.832298,-0.000469,0.588192,-2.210818e-04,0.415758,-0.000053,0.293948
2,-1.797355,-6.714585e-17,-1.270922,-1.503918e-17,-0.898677,-8.230210e-18,-0.635461,-0.193655,-2.483131e-07,-0.136935,...,0.0,0.0,0.0,0.832246,-0.000521,0.588118,4.903912e-18,0.415862,0.000157,0.294171
3,-1.797355,-6.714585e-17,-1.270922,-1.503918e-17,-0.898677,-8.230210e-18,-0.635461,-0.193655,-3.724696e-07,-0.136935,...,0.0,0.0,0.0,0.832246,-0.000313,0.588266,2.947758e-04,0.416175,0.000155,0.294390
4,-1.797355,-6.714585e-17,-1.270922,-1.503918e-17,-0.898677,-8.230210e-18,-0.635461,-0.193655,-4.966262e-07,-0.136935,...,0.0,0.0,0.0,0.832167,-0.000079,0.588375,2.571100e-04,0.416226,-0.000104,0.294243


In [6]:
# Configure with an offset
offset = 20
config_offset = extract_wavelet_features.WaveletConfig(
    level=LEVEL, overlap=0.875, offset=offset
)
extractor_offset = extract_wavelet_features.ExtractWaveletFeatures(config_offset)
extractor_offset.is_windowed = True
features_offset = extractor_offset(windowed_data)

features_offset.head()

,var1_A3,var1_D3,var1_A2,var1_D2,var1_A1,var1_D1,var1_A0,var2_A3,var2_D3,var2_A2,...,var21_A1,var21_D1,var21_A0,var22_A3,var22_D3,var22_A2,var22_D2,var22_A1,var22_D1,var22_A0
0,-1.797355,-6.714585e-17,-1.270922,-1.503918e-17,-0.898677,-8.230210e-18,-0.635461,-0.193657,-4.966262e-07,-0.136936,...,0.0,0.0,0.0,0.831804,-0.000234,0.588009,0.000074,0.415837,0.000025,0.294059
1,-1.797355,-6.714585e-17,-1.270922,-1.503918e-17,-0.898677,-8.230210e-18,-0.635461,-0.193657,-3.724696e-07,-0.136936,...,0.0,0.0,0.0,0.831856,-0.000182,0.588082,0.000072,0.415888,0.000025,0.294095
2,-1.797355,-6.714585e-17,-1.270922,-1.503918e-17,-0.898677,-8.230210e-18,-0.635461,-0.193657,-2.483131e-07,-0.136936,...,0.0,0.0,0.0,0.831777,-0.000131,0.588063,-0.000020,0.415809,-0.000104,0.293948
3,-1.797355,-6.714585e-17,-1.270922,-1.503918e-17,-0.898677,-8.230210e-18,-0.635461,-0.193657,-1.241565e-07,-0.136936,...,0.0,0.0,0.0,0.831569,-0.000080,0.587951,-0.000203,0.415601,-0.000104,0.293800
4,-1.797355,-6.714585e-17,-1.270922,-1.503918e-17,-0.898677,-8.230210e-18,-0.635461,-0.193658,-1.241565e-07,-0.136937,...,0.0,0.0,0.0,0.831464,-0.000105,0.587860,-0.000183,0.415550,0.000053,0.293876


## Statistical Feature Extraction

This is the most common approach to feature extraction for time-series data. The `ExtractStatisticalFeatures` class takes the pre-windowed data and calculates a set of standard statistical descriptors for each window. These features summarize the shape and distribution of the data within that specific time segment.

The features extracted are:
* **`mean`, `std`**: Describe the central tendency and dispersion (volatility).
* **`skew`, `kurtosis`**: Describe the shape of the distribution (asymmetry and presence of outliers).
* **`min`, `1qrt`, `med`, `3qrt`, `max`**: Provide a summary of the distribution through quartiles.

Since our data is already windowed from the previous step, we can reuse the `windowed_data` `DataFrame` directly.

In [7]:
# Configure the statistical feature extractor
statistical_config = StatisticalConfig(
    window_size=WINDOW_SIZE, overlap=OVERLAP
)

# Instantiate the extractor
statistical_feature_extractor = extract_statistical_features.ExtractStatisticalFeatures(
    statistical_config
)
statistical_feature_extractor.is_windowed = True

# Run the feature extraction step by calling the instance
statistical_features = statistical_feature_extractor(windowed_data)

# Display the results
print("Shape of the final statistical features:", statistical_features.shape)
print("\nColumns are named 'var<index>_<feature>', and the label column is preserved.")
statistical_features.head()

Shape of the final statistical features: (561714, 198)

Columns are named 'var<index>_<feature>', and the label column is preserved.


,var1_mean,var1_std,var1_skew,var1_kurt,var1_min,var1_1qrt,var1_med,var1_3qrt,var1_max,var2_mean,...,var21_max,var22_mean,var22_std,var22_skew,var22_kurt,var22_min,var22_1qrt,var22_med,var22_3qrt,var22_max
0,-0.635461,0.0,0.0,0.0,-0.635461,-0.635461,-0.635461,-0.635461,-0.635461,-0.068467,...,0.0,0.294299,0.000164,-0.299949,-0.983426,0.294023,0.294206,0.294318,0.294409,0.294537
1,-0.635461,0.0,0.0,0.0,-0.635461,-0.635461,-0.635461,-0.635461,-0.635461,-0.068467,...,0.0,0.294262,0.000201,-0.267462,-1.354669,0.293948,0.294077,0.294318,0.294409,0.294537
2,-0.635461,0.0,0.0,0.0,-0.635461,-0.635461,-0.635461,-0.635461,-0.635461,-0.068467,...,0.0,0.294243,0.000202,-0.004057,-1.426620,0.293948,0.294077,0.294244,0.294409,0.294537
3,-0.635461,0.0,0.0,0.0,-0.635461,-0.635461,-0.635461,-0.635461,-0.635461,-0.068467,...,0.0,0.294243,0.000202,-0.004057,-1.426620,0.293948,0.294077,0.294244,0.294409,0.294537
4,-0.635461,0.0,0.0,0.0,-0.635461,-0.635461,-0.635461,-0.635461,-0.635461,-0.068467,...,0.0,0.294216,0.000184,0.232607,-0.977814,0.293948,0.294077,0.294207,0.294336,0.294537


## Exponentially Weighted Statistical Feature Extraction

The `ExtractEWStatisticalFeatures` class provides a specialized version of the standard statistical features. The "EW" stands for **Exponentially Weighted**.

In this method, not all data points in a window are treated equally. Instead, more recent data points are given progressively higher weight than older points. The rate at which the importance of older data "decays" is controlled by the `decay` parameter.

This is particularly useful in scenarios where the most recent behavior within a window is more predictive of the outcome than the behavior at the beginning of the window. It creates features that are more sensitive to the latest changes in the signal.

Again, we will use the same `windowed_data` `DataFrame` as input.

In [8]:
# Configure the EW statistical feature extractor
# The decay parameter is specific to this class and controls the weighting
ew_statistical_config = EWStatisticalConfig(
    window_size=WINDOW_SIZE, overlap=OVERLAP, decay=0.9
)

# Instantiate the extractor
ew_feature_extractor = (
    extract_exponential_statistics_features.ExtractEWStatisticalFeatures(
        ew_statistical_config
    )
)
ew_feature_extractor.is_windowed = True

# Run the feature extraction step by calling the instance
ew_statistical_features = ew_feature_extractor(windowed_data)

# Display the results
print("Shape of the final EW statistical features:", ew_statistical_features.shape)
print(
    "\nColumns are named 'var<index>_ew_<feature>', and the label column is preserved."
)
ew_statistical_features.head()

KeyboardInterrupt: 